# Obsah cvicenia

> ## Chybajuce hodnoty

> ## Vychylene hodnoty

> ## Normalizacia udajov

> ## Transformacia numerickych atributov na kategoricke

> ## Transformacia kategorickych udajov na numericke

> ## Vytvaranie atributov kombinovanim

Na predspracovanie odporucame vyuzit kniznicu `scikit-learn`. Moznosti, ktore na predspracovanie ponuka, najdete tu: http://scikit-learn.org/stable/modules/preprocessing.html

Vycerpavajuci zoznam mnozstva dalsich veci, co sa da robit na pripravu dat a na feature engineering najdete tu: http://www.datasciencecentral.com/profiles/blogs/feature-engineering-data-scientist-s-secret-sauce-1

Pekne spracované je predspracovanie a čistenie dát aj tu (akurát s príkladmi v R, nie pythone): https://cran.r-project.org/doc/contrib/de_Jonge+van_der_Loo-Introduction_to_data_cleaning_with_R.pdf Zaujímavá je hlavne časť k chýbajúcim a vychýleným hodnotám.

# Chýbajuce hodnoty

Existujú viaceré stratégie nahrádzania chýbajúcich hodnôt. Medzi najčastejšie patria:

* nahradenie chýbajúcej hodnoty mediánom
* nahradenie chýbajúcej hodnoty priemerom
* nahradenie chýbajúcej hodnoty pomerom ku korelovanému atribútu
* nahradenie chýbajúcej hodnoty priemerom segmentu
* nahradenie chýbajúcej hodnoty pomocou lineárnej regresie
* nahradenie chýbajúcej hodnoty pomocou algoritmu k-najbližších
susedov

Užitočné metódy/knižnice na prácu s chýbajúcimi hodnotami:

* `dropna`, `fillna`
* `sklearn.preprocessing.Imputer`: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html
* `fancyimpute`: https://github.com/iskandr/fancyimpute

Ďalšie informácie k chýbajúcim hodnotám môžete nájsť tu: https://medium.com/ibm-data-science-experience/missing-data-conundrum-exploration-and-imputation-techniques-9f40abe0fd87

# Vychýlené hodnoty

Aj pri riešení vychýlených hodnôt možno aplikovať viaceré stratégie:

* odstránenie vychýlených (odľahlých) pozorovaní
* nahradenie vychýlenej hodnoty hraničnými hodnotami rozdelenia (5 percentilom, resp. 95 percentilom)
* transformácia atribútu s vychýlenými hodnotami pomocou zvolenej funkcie (logaritmus, odmocnina a pod.)

*Pozor*: Pri rozhodovaní, čo je, a čo nie je vychýlená hodnota, treba brať do úvahy rozdelenie premennej. Určenie vychýlených hodnôt pomocou boxplotu funguje pre normálne rozdelenie. Ak má premenná iné rozdelenie, môžeme ju transformovať (normalizovať).

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn import preprocessing

plt.rcParams['figure.figsize'] = 6, 4
from IPython.display import Image

# Normalizacia udajov

In [ ]:
data_file = "./data/x_train.csv"
challenge_data = pd.read_csv(data_file, nrows=1000, sep=',', header=None)
challenge_data.head()

In [ ]:
challenge_data.info()

In [ ]:
# seaborn.pairplot(netcla) # toto nije dobry napad. Tych atributov je strasne vela a kazdy s kazdym je dost velka matica
# plt.rcParams['figure.figsize'] = 18, 12
challenge_data.hist(figsize=(18, 12))

In [ ]:
index = 20
pom = challenge_data[index].hist(bins=50)
pom.set_title(index)

In [ ]:
challenge_data[index].plot(kind='box')

In [ ]:
challenge_data[20].min()

In [ ]:
index = 20
transformed = np.log(challenge_data[index]+2)
pom = pd.Series(transformed).hist(bins=50)
pom.set_title("index: %d (log transform)" % index)

Lepsie ako logaritmus je pouzit Box-Cox. Box-Cox sa dokaze postarat o to, aby sa vysledne rozelenie podobalo normalnemu. Viac info o tom, ako Box-Cox funguje, najdete tu: http://onlinestatbook.com/2/transformations/box-cox.html

In [ ]:
from scipy.stats import boxcox

In [ ]:
index = 20
transformed, att = boxcox(challenge_data[index]+2)# boxcox vrati transformovane data a parametre transformacie. Tie viem zafixovat a v tom pripade mi to vrati len transformovane data
pom = pd.Series(transformed).hist(bins=50)
pom.set_title("index: %d (box-cox)" % index)

In [ ]:
pd.Series(transformed).plot(kind='box')

In [ ]:
# z-normalization: shift = mean, scale = std
# 0-1 normalization: shift = min, scale =  max - min
# toto iste robi StandardScaler pre vsetky atributy. Pozor ale na power law
# ak pouzijete RobustScaler, tak si viete poradit aj s outliermi (pouziva rozsah 1st quartile (25th quantile) and the 3rd quartile (75th quantile))
def normalization(data, shift, scale): 
    return (np.array(data) - float(shift))/scale

In [ ]:
z_transformed = normalization(transformed, np.mean(transformed), np.std(transformed))
pom = pd.Series(z_transformed).hist(bins=50)
pom.set_title('throughput (Z-normalization)')

In [ ]:
# sklearn.preprocessing
preprocessing.scale(transformed)
pom = pd.Series(z_transformed).hist(bins=50)
pom.set_title('throughput (Z-normalization)')

# Transformacia numerickych atributov na kategoricke - diskretizacia

Pre niektore algoritmy ciselne data nie su uplne vhodne. Napriklad taky Naivny Bayes potrebuje kategoricke hodnoty. Tie sa daju vytvorit zaradenim numerickych hodnot do intervalov.

Intervaly su bud manualne nastavene, alebo take, aby ich sirka bola rovnaka (Equal width binning/discretization) alebo pocty prvkov (Equal frequency binning/discretization), ktore do nich spadaju boli priblizne rovnake. 

## Rovnaka sirka intervalov

In [ ]:
bin_count = 10
bins = np.linspace(z_transformed.min(), z_transformed.max(), bin_count)
ew_digitized = np.digitize(z_transformed, bins)
ew_digitized[:10]

In [ ]:
bins

In [ ]:
z_transformed[:10]

In [ ]:
plt.bar(*np.unique(ew_digitized, return_counts=True))

In [ ]:
bins

pri rovnako sirokych intervaloch som rovnomerne rozdelil intrval medzi maximalnou a minimalnou hodnotou. Nic mi nebrani definovat si vlastne, rozne siroke intervaly a spravit z toho pristup pouzivajuci manualne definovane intervaly

## Rovnaka pocetnost intervalov

Chceme, aby boli intervali priblizne rovnako pocetne. Predpokladajme, ze sme uz spravili nejake transformacie dat, a tie teraz priblizne dodrzuju normalne rozdelenie

In [ ]:
# toto je kod na vytvorenie fancy grafu, ktory nemusite podrobne studovat, ak sa vam nechce. Dolezity je vysledok, aby som na nom vysvetlil princip transformacie

from scipy.stats import norm

def breakpoints(alphabetSize):
        return list(map(norm.ppf, np.linspace(0,1,alphabetSize+1)[1:-1]))
    
def interval_centres(alphabetSize):
    return breakpoints(alphabetSize * 2)[::2]
    
def adjust_spines(ax, spines):
    for loc, spine in ax.spines.items():
        if loc in spines:
            spine.set_position(('outward', 10))  # outward by 10 points
            spine.set_smart_bounds(True)
        else:
            spine.set_color('none')  # don't draw spine

    # turn off ticks where there is no spine
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        # no yaxis ticks
        ax.yaxis.set_ticks([])

    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
        # no xaxis ticks
        ax.xaxis.set_ticks([])

plt.rcParams['figure.figsize'] = 9, 3

fig, ax = plt.subplots(1, 1)

symbol_num = 5
brp = breakpoints(symbol_num)
centres = interval_centres(symbol_num)
lw = 1.4

plt.vlines(brp, 0.005, list(map(norm.pdf, brp)), color='red', linewidth=lw)

x = np.linspace(norm.ppf(0.0001), norm.ppf(0.9999), 100)
ax.plot(x, norm.pdf(x), '-', color='black', linewidth=lw)

hatched2 = np.linspace(-4, brp[0], 100)
ax.fill_between(hatched2, norm.pdf(hatched2), facecolor="none", hatch="///", edgecolor="black", linewidth=0.0, alpha=0.5)

hatched = np.linspace(brp[0], brp[1], 100)
ax.fill_between(hatched, norm.pdf(hatched), facecolor="none", hatch='\\\\\\', edgecolor="black", linewidth=0.0, alpha=0.5)

adjust_spines(ax, ['left', 'bottom'])
# adjust_spines(ax, ['bottom'])

letters = 'ABCDEFGHIJKLMNOPQRSTUVXYZ'
for i in range(symbol_num):
    ax.annotate(letters[i], xy=(centres[i], 0.05), horizontalalignment='center', verticalalignment='middle', backgroundcolor='w', 
                fontsize='15', color='black', bbox=dict(color='white', alpha=0.7))

ax.set_ylim([0, 0.45])

kazdy interval ma presne rovnaky obsah pod kryvkou. To znamena, ze ak podla tychto intervalov rozdelime data z normalneho rozdelenia do roznych symbolov / kategorickych hodnot, tak pocetnost kazdej z tychto hodnot bude priblyzne rovnaky

In [ ]:
plt.rcParams['figure.figsize'] = 6, 4

In [ ]:
def create_breakpoints(alphabetSize):
        brkp_list = list(map(norm.ppf, np.linspace(0,1,alphabetSize+1)))
        return [(brkp_list[i], brkp_list[i+1]) for i in range(alphabetSize)]
create_breakpoints(5)

In [ ]:
def interval_transform(data, breakpoints):
    def get_interval(value, breakpoints):
        for index in range(len(breakpoints)):
            breakpoint = breakpoints[index]
            if value > breakpoint[0] and value <= breakpoint[1]:
                return index
        return len(breakpoints)
    
    return pd.Series([get_interval(value, breakpoints) for value in data])

In [ ]:
interval_number = 10
interval_transformed = interval_transform(z_transformed, create_breakpoints(interval_number))
pom = pd.Series(interval_transformed).hist(bins=interval_number)
pom.set_title('transformed attribute (Interval normalization)')

Kedze sme nemali uplne pekne normalne rozdelenie, tak ani intervaly niesu uplne rovnomerne pocetne, ale niesme daleko

## Nieco podobne by sa dalo dosiahnut aj pomocou percentilov v pripade, ak nemame normalizovane data

Toto by sme vedeli pustat na hociake rozdelenie dat a nemuseli by sme sa spoliehat na nejake teoreticke rozdelenie, ktore by mali tie nase data splnat. A dokonca bude ten zapis aj jednoduchsi :)

In [ ]:
data = challenge_data[20] # tuna nepotrebujeme transformovat data, ale ak by sme, tak nam to nijak neprekaza

bin_count = 10
percentiles = np.linspace(0, 100, bin_count, endpoint = False)
bins = list(map(lambda x: np.percentile(data, x), percentiles))
ef_digitized = np.digitize(data, bins)
ef_digitized[:10]

In [ ]:
percentiles

In [ ]:
bins

In [ ]:
plt.bar(*np.unique(ef_digitized, return_counts=True))

### Co sa stane ak tu istu transformaciu pustime na testovacich datach?

In [ ]:
challenge_data_test = pd.read_csv("./data/x_test.csv", nrows=1000, sep=',', header=None)
test_data = challenge_data_test[20]

# nezmenim biny, cize pouzivam transformaciu, ktoru som natrenoval na trenovacich datach. Na toto si treba dat pozor. Nikdy transformaciu nemozem menit pre trenovacie a pre testovacie data
ef_digitized_test = np.digitize(test_data, bins)
plt.bar(*np.unique(ef_digitized_test, return_counts=True))

# Transformacia numerickych dat na binarne

Toto je jednoduche, potrebujete si len zobrat nejaku hranicu a vsetko, co je pod nou bude 0 a nad nou 1

In [ ]:
binarizer = preprocessing.Binarizer(0) # v tomto pripade je hranica 0
binary = binarizer.transform(z_transformed.reshape(1, -1))
pd.Series(binary[0]).hist()

# Transformacia kategorickych udajov na numericke

http://www.willmcginnis.com/2015/11/29/beyond-one-hot-an-exploration-of-categorical-variables/

https://github.com/wdm0006/categorical_encoding

https://stats.idre.ucla.edu/r/library/r-library-contrast-coding-systems-for-categorical-variables/

### Existuje viacero sposobov ako transformovat kategoricku hodnotu na cislo 

1. Ordinal - priradzovanie cisel postupne roznym hodnotam
2. One-hot - z kazdej kategorie vznikne stlpec s hodnotou 1 v tych riadkoch, ktore boli nastavene na tuto hodnotu. inde 0
3. Binary - zoberie sa ordinal, zakoduju sa tie cisla ako binarne, kazda cislica binarneho cisla je pouzita ako stlpec a tam kde bola na zodpovedajucom mieste 1, tam bude v stlpci 1 a inak 0
4. Sum - porovnava sa priemer zavislej premennej na riadokch jednej skupiny oproti priemeru zavislej premennej na celej datovej sade
5. Helmert - velmi podobne ako Sum, len jedinecnost categorickej hodnoty je dana inou kombinaciou hodnot
6. BackwardDifferenceEncoder - velmi podobne ako Sum, len jedinecnost categorickej hodnoty je dana inou kombinaciou hodnot
7. Polynomial - trenuje koeficienty ploynomialnej regresie rozneho stupna, ktore sa daju pouzit na regresiu zavislej premennej (neviem aky to ma zmysel pre maly pocet roznych hodnot) (treba ordinalne premenne)
8. Hash - zahashuje string kategorickej premennej a moduluje ho poctom roznych hodnot. Je v sklearn http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html

Zaujimave su asi len prve 4 a cislo 7, ostatne su len obmeny tychto s tym, ze menia rozdelenia hodnot

In [ ]:
titanic = pd.read_csv('data/titanic/train.csv')
titanic.head()

In [ ]:
# pip install category_encoders
import category_encoders as ce

# encoder = ce.OrdinalEncoder()
# encoder = ce.BinaryEncoder()
# encoder = ce.PolynomialEncoder() 
# encoder = ce.HashingEncoder()

# encoder = ce.HelmertEncoder()
encoder = ce.OneHotEncoder() # tento je aj v sklearn.preprocessing.OneHotEncoder, tu su ale vsetky a maju rovnaky predpis
# encoder = ce.BackwardDifferenceEncoder()
# encoder = ce.SumEncoder()

In [ ]:
titanic_X = titanic.dropna().reindex(columns=[x for x in titanic.columns.values if x != 'Survived'])
titanic_y = titanic.dropna().reindex(columns=['Survived'])

In [ ]:
encoder.fit(titanic_X, titanic_X.Fare)
titanic_categorical = encoder.transform(titanic_X)
titanic_categorical.head()

toto sme asi uplne nechceli. Treba si davat pozor na to kolko roznych hodnot ma kategoricka premenna. Tu sme sa pokusili zakodovat meno, ktore ma velmi vela hodnot a teda nam vzniklo velmi vela roznych stlpcov. Vo vseobecnosti to nieje dobry napad

# Vytvaranie atributov kombinovanim

### Kombinovanie numerickych a kategorickych dat

In [ ]:
titanic.head()

In [ ]:
mean_age = titanic.Age.mean()
mean_age

In [ ]:
titanic['mean_age_diff'] = pd.Series('', index=titanic.index)
titanic[['mean_age_diff']] = titanic.Age.apply(lambda x: x-mean_age)
titanic.head()

In [ ]:
mean_attributes_by_sex = titanic.groupby('Sex').mean()
# mean_attributes_by_sex.loc['female','Age']
mean_attributes_by_sex

In [ ]:
titanic['mean_age_diff_per_sex'] = pd.Series('', index=titanic.index)
titanic[['mean_age_diff_per_sex']] = titanic.apply(lambda x: x['Age']-mean_attributes_by_sex.loc[x['Sex'],'Age'], axis=1)
titanic.head()

In [ ]:
combined = titanic.dropna()._get_numeric_data()
combined.head()

### Polynomialne kombinovanie

In [ ]:
X = np.arange(6).reshape(3, 2)
X

In [ ]:
poly = preprocessing.PolynomialFeatures(3)
poly.fit_transform(X) # vytvorenie polynomialnych kombinacii

In [ ]:
# nevieme pouzivat prazdne hodnoty pri vypocte polynomialnych vlastnosti
titanic_X = titanic.dropna().reindex(columns=[x for x in titanic.columns.values if x != 'Survived']).reset_index(drop=True)
titanic_y = titanic.dropna().reindex(columns=['Survived']).reset_index(drop=True)

In [ ]:
poly = preprocessing.PolynomialFeatures(2) # pozor na prilis velke cislo. Vela atributov a pretrenovanie
# polynomial_titanic = poly.fit_transform(titanic_X)
polynomial_titanic = poly.fit_transform(titanic_X._get_numeric_data())

In [ ]:
polynomial = pd.DataFrame(polynomial_titanic)
polynomial.head()

In [ ]:
titanic_X._get_numeric_data().head()

ak by ste chceli zachovat rozumne nazvy stlpcov: http://stackoverflow.com/questions/36728287/sklearn-preprocessing-polynomialfeatures-how-to-keep-column-names-headers-of

In [ ]:
polynomial['Survived'] = titanic_y
polynomial.head()

In [ ]:
from sklearn.model_selection import cross_val_score
# clf = LogisticRegression()
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)

cross_validation_results = cross_val_score(clf, polynomial[polynomial.columns[polynomial.columns != 'Survived']], polynomial['Survived'], cv=6)
(cross_validation_results.mean(), cross_validation_results.std())

In [ ]:
cross_validation_results

In [ ]:
original = titanic.dropna()._get_numeric_data()
original.head()

In [ ]:
len(original)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)


cross_validation_results = cross_val_score(clf, original[original.columns[original.columns != 'Survived']], original['Survived'], cv=6)
(cross_validation_results.mean(), cross_validation_results.std())

# Varovanie k vyrabaniu a transformacii atributov

* Ked budete robit hociake operacie nad datami, tak si dajte pozor aby ste robili rovnake operacie nad trenovacou aj testovacou sadou (transformacie atributov, vytvaranie novych, filtrovanie ...)
* Dajte si pozor aby vam do trenovania nepretiekli udaje z buducnosti
> * ked idem napriklad normalizovat nieco priemerom, tak hodnotu priemeru pocitam len nad trenovacimi datami a nie nad vsetkymi 
> * Pri tomto vedia velmi pomoct tzv. Pipeliny
>> http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

* ked budete normalizovat udaje, tak na normalizovanie testovacej vzorky pouzite koeficienty z trenovacej vzorky